<a href="https://colab.research.google.com/github/Hongbi-Kim/Writing_lyrics/blob/master/GPT3_%EA%B2%B0%EA%B3%BC_%EB%8F%8C%EB%A0%A4%EB%B3%B4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install transformers

- gpt3_1 : tokenizer 250, 300으로 학습
- gpt3_2 : tokenizer 350으로 학습
- gpt3_5000sample : 5000개 데이터, tokenizer 350으로 학습
- gpt3_10000개_350_epoch7
- gpt3_10000_mx350_ep7

In [25]:
# 구글 드라이브에서 모델, 토크나이저 불러오기
import os
import time
import datetime
from google.colab import drive

import pandas as pd
import seaborn as sns
import numpy as np
import random
import re

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

# gpt3-kor-small_based_on_gpt2
from transformers import BertTokenizerFast, GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/5조_파이널PJT/코드/model_result/gpt3_topic')
tokenizer_gpt3 = BertTokenizerFast.from_pretrained('/content/drive/MyDrive/5조_파이널PJT/코드/model_result/gpt3_topic')
input_ids = tokenizer_gpt3.encode("text to tokenize")[1:]  # remove cls token

In [26]:
device = torch.device("cuda")
model.cuda()

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
model = model.to(device)

In [27]:
model.eval()
def lyric_generator(gen, lyric):
  prompt = f"<|startoftext|> <{gen}> {lyric}"
  generated = torch.tensor(tokenizer_gpt3.encode(prompt)[1:]).unsqueeze(0)
  generated = generated.to(device)

  print(generated)

  sample_outputs = model.generate(
                                  generated, 
                                  #bos_token_id=random.randint(1,30000),
                                  do_sample=True,   
                                  top_k=50, 
                                  max_length = 350,
                                  #max_length = len(lyric) + 350,
                                  top_p=0.95, 
                                  num_return_sequences=1,
                                  repetition_penalty=1.1
                                  )

  for i, sample_output in enumerate(sample_outputs):
    # result = "{}: {}".format(i, tokenizer_gpt3.decode(sample_output, skip_special_tokens=True))
    result = tokenizer_gpt3.decode(sample_output, skip_special_tokens=True)
    result = re.sub(r"<br>", '\n', result)
    # re.sub(r'[a-z]','',result.split('\n')[-1])
    # result = re.sub(r'[a-z]','',result.split('\n')[-1])   
    # result = result[len(lyric)+1:]
    # print(re.sub(r'[a-z]','',result))
    #print(result)
  return result

In [28]:
def decide_regen(input):
  val = []
  for i in input.split('<br>'):
  #for i in input.split('\n'):
    val.append(str(bool(re.search(r'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]',i))))

  eng = []
  space = []
  for char in input:
    if re.sub(r'[a-z]','',char) == "":
      eng += char
    elif char == " ":
      space += char
  eng_ratio = len(eng)/(len(input)-len(space))

  if (len(set(val[1:])) == 1) & ('False' in set(val[1:])) == True:
    print((len(set(val[1:])) == 1) & ('False' in set(val[1:])))
    print(input)
    output = 'regenerate'
  elif eng_ratio > 0.7:
    print(eng_ratio)
    print(input)
    output = 'regenerate'
  else:
    print(eng_ratio)
    output = 'pass'
  
  return output

In [29]:
def remove_lastEng(a):
  # 마지막 줄에 영어만 있다면
  if re.sub(r'[a-z\n]','',a.split('\n')[-1]).strip() == "" :
    b = a.split('\n')[:-1]
    c = ""
    for i in b:
      if i == ' ':
        i = re.sub(r" ", "\n \n", i)
      c += i
      c = re.sub(r"  ", "\n", c)
      output = c
  else:
    output = a.replace('<br>','\n')
  print(output)
  return 

In [30]:
def ai_lyrics(gen, lyric):
  result = lyric_generator(gen, lyric)
  if decide_regen(result) == 'regenerate':
    print('재생성이 필요합니다.')
    # output = lyric_generator(gen, lyric)
    return ai_lyrics(gen, lyric)
  else:
    return remove_lastEng(result)
  #print(output)
  #return

In [37]:
ai_lyrics('댄스topic4','똑같이 반복되는 하루 지긋지긋해')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42012, 20320, 16366, 14106, 14298,  6266,  9110,  8038,  9110,
          8010,     3]], device='cuda:0')
0.0
똑같이 반복되는 하루 지긋지긋해 
 시간 있으면 전화하고 나올 거야 
 오늘 하루는 내가 만든 노래를 불러볼래 
 해가 지면 톡 해줘요 
 오늘은 그냥 나랑 놀자 
 나랑 같이 놀자 
 내가 만든 노래에 흥얼거려봐 
 나와 같이 놀자 
 나와 같이 놀자 
 내가 만든 노래에 빠져서 춤을 춰요 
 노래는 우리가 만든 노래 
 이 음악이 돈이 되는 순간 행복할 거야 
 내게는 이 노래가 평생 아껴줄 거야 
 내가 가진 노래로 흥얼거릴 거야 
 함께하면 더 빠르게 맘이 벅차오르고 
 너를 보면 벅차오르고 
 너도 나를 본다면 이 순간을 잊지 않을 거야 
 나처럼 나처럼 리듬을 타볼 거야 
 나와 같이 놀자 나랑 같이 놀자 
 내가 만든 노래에 흥얼거려봐 
 나와 같이 놀자 나랑 같이 놀자 
 내가 만든 노래에 빠져서 춤을 춰요 
 노래는 우리가 만든 노래 
 이 음악이 돈이 되는 순간 행복할 거야 
 나와 같이 놀자 나랑 같이 놀자 
 나랑 같이 놀자 나랑 같이 놀자 
 나랑 같이 놀자 나랑 같이 놀자 
 나랑 같이 놀자 나랑 같이 놀자 
 나랑 같이 놀자 나랑 같이 놀자 
 나랑 같이 놀자 나랑 같이 놀자 
 나랑 같이 놀자 나랑 같이 놀자 
 나랑 같이 놀자 나랑 같이 놀자


In [19]:

print('똑같이 반복되는 하루하루 저 멀리 들리는\n작은 음악소릴 따라 걸으면\n어느새 너와 내가 같은 꿈을 꾼 거 같아\n왜 이렇게 이 모든 게 꿈만같아\ni wanna feel\n지금 우리 둘만의 리듬을 타고\ngo to the moonlight\n별빛은 우릴 비추고\ndont let it go away\n그 어떤것도 우리 맘을 막을 수는 없어\n너와 함께 이 세상을\n더 크게 만들어 볼래\neveryday everynight\n너와나 함께해 매일 새롭게\n우리 함께할 우리의 미래를 펼쳐갈게\n이 짙은 어둠이 사라지고\n이젠 함께 하는 거야 always\n저 파란 하늘 아래 이 노래는\noh eh oh oh\n밝은 아침이 되면\n서로 어깰 두드리며 환하게 웃어줘\n더 멋진 날을 위해 우린 달려왔어\nlets get party\n다함께 시작해볼까\ncan we stay in the moment\ncan you be yeah uh oh\nnananananana\ncan you be yeah uh oh\nnow lets bring on back turn it up\ncome on overdrive and run and drug\n너와 나 이 선을 넘어\nwe can stop\n다 같이 we can rock it on\n멈추지 않고 fly high\n저 높은 하늘을 향해서 크게 소리쳐봐\n우리가 바라는 멋진 미래로\n꿈이 이뤄질 아름다운 날들\n우리의 발걸음\n더 가볍게 더 신나게\n이 짙은 어둠이 사라져\n이젠 함께 하는 거야 always\n저 파란 하늘 아래 이 노래는 ')

똑같이 반복되는 하루하루 저 멀리 들리는
작은 음악소릴 따라 걸으면
어느새 너와 내가 같은 꿈을 꾼 거 같아
왜 이렇게 이 모든 게 꿈만같아
i wanna feel
지금 우리 둘만의 리듬을 타고
go to the moonlight
별빛은 우릴 비추고
dont let it go away
그 어떤것도 우리 맘을 막을 수는 없어
너와 함께 이 세상을
더 크게 만들어 볼래
everyday everynight
너와나 함께해 매일 새롭게
우리 함께할 우리의 미래를 펼쳐갈게
이 짙은 어둠이 사라지고
이젠 함께 하는 거야 always
저 파란 하늘 아래 이 노래는
oh eh oh oh
밝은 아침이 되면
서로 어깰 두드리며 환하게 웃어줘
더 멋진 날을 위해 우린 달려왔어
lets get party
다함께 시작해볼까
can we stay in the moment
can you be yeah uh oh
nananananana
can you be yeah uh oh
now lets bring on back turn it up
come on overdrive and run and drug
너와 나 이 선을 넘어
we can stop
다 같이 we can rock it on
멈추지 않고 fly high
저 높은 하늘을 향해서 크게 소리쳐봐
우리가 바라는 멋진 미래로
꿈이 이뤄질 아름다운 날들
우리의 발걸음
더 가볍게 더 신나게
이 짙은 어둠이 사라져
이젠 함께 하는 거야 always
저 파란 하늘 아래 이 노래는 


In [ ]:
ai_lyrics('댄스topic5', '매일 강아지랑 춤추는')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42013, 14759, 14971,  8202,  6748, 33408,     3]],
       device='cuda:0')
매일 강아지랑 춤추는 
 모두 잠든 새벽 고요한 강둑에 
 수양버들이 아 아 
 밤 하늘에 흐르는 별이 빛나는 밤에 
 저 별들의 노래 달콤이 전해지네 
 자 소원을 들어줘 그리고 춤을 춰봐 
 함께 가는 거야 우리 노래 부르자 
 함께 보는 거야 영원을 향해 날아보자 
 자 이제 약속해 시간이 됐어 
 소리 지르고 어서 따라와 
 우리는 저멀리로 아 아 
 우리의 노래가 시작되면 
 모두가 놀라 귀척이나 달려가 
 노래 불러라 소리 질러 
 자 준비됐음 잘 들었으니 
 나와 함께 놀자 
 하하하하하 하늘을 날아가는 새처럼 
 우리 들판에 물들게 
 까치발 들고서 라랄랄라라라 
 랄랄라라라라라 라랄라 라랄라 
 자 이제는 준비 됐어 
 자 마음됨대로 함께뛰어 놀아보자 
 자 원 투히기 원 투 쓰리 포 
 자 준비됐음 시작해보자 
 자 목소리 위 
 랄랄라라라라라 라랄랄라라라 
 랄랄라라라라라 라랄라 라랄라 
 자 준비됐음 시작해보자 
 자 이제부터 시작해보자


In [ ]:
lyric_generator('발라드', '매일 강아지랑 산책해')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42003, 14759, 14971,  8202, 16780,  8010,     3]],
       device='cuda:0')
0: 매일 강아지랑 산책해 
 용기 내 전불 걸 수 있는 
 용길 줘요 함께해요 
 이 넓은 곳에서 우리 둘의 멋진 미래를 봐요 
 함께 달려갈래요 서로 손을 잡고서 
 우리 모두 함께 달려갈래요 우리 모두 
 별이 뜰 때까지 lets go 함께해요 
 별이 뜰 때까지 
 i love you i need you baby 
 별이 뜰 때까지 lets go 함께해요 
 너만의 빛나는 별을 찾아갈래 
 널 위한 별자리를 만들래 
 난 너만의 반짝이는 별이 될래 
 너만의 빛으로 밝게 빛나줘 
 너만의 환한 빛으로 
 밝게 빛나줘 
 너만의 빛나는 별을 찾을래 
 널 위한 별자리를 만들래 
 넌 날 깨워내는 저 빛의 마법사 
 저 높은 곳까지 올라 갈 것 같아 
 내 손에 잡힐 듯 말 듯이 잡히지가 않아 
 너의 주위를 공전하는 저 달이 
 이 어두운 밤하늘에 뜨는 별처럼 
 내가 지켜줄게요 
 같이 달려 갈래요 서로의 손 잡고서 
 너와 나 이렇게 둘이 저 빛나는 하늘을 향해가요 
 이 넓은 곳에서 우리 둘의 멋진 미래를 봐요 
 함께 달려갈래요 서로 손을 잡고서 
 우리 모두 함께 달려갈래요 우리 모두 
 별이 뜰 때까지 lets go 함께해요 
 별이 뜰 때까지 
 i love you i need you baby 
 별이 뜰 때까지 lets go 함께해요 
 함께 날아갈래요 
 ah ah ah ah ah ah ah 
 ah ah ah ah ah ah ah 
 같은 시간에 같은 시간에 함께라면 
 lets get a dream come true 
 함께 즐겨볼래요 hey 
 함께 날아갈래요 
 ah a




In [ ]:
len('주말마다 강아지랑 산책해요<br> 강아지는 너무 귀엽고 뛰어다녀요 <br> ')

36

In [ ]:
lyric_generator('댄스', '주말마다 강아지랑 산책해요<br> 강아지는 너무 귀엽고 뛰어다녀요')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 15263, 14304, 14971,  8202, 16780, 14088, 42005, 39847,
         14000, 21752, 16357, 22481,  8055,     3]], device='cuda:0')
0: 주말마다 강아지랑 산책해요 
 강아지는 너무 귀엽고 뛰어다녀요 
 바다로 갈래 어디로 갈지 몰라서 
 그 동안 못한 사랑 다시 찾았죠 
 이제는 더 이상 기다리기 지겨워지면 
 늦기전에 떠나요 멀리 떠나요 
 푸른 바다로 신나는 여름 속으로 
 푸른 바다로 신나는 여름 속으로 
 언제나 함께 있어줘요 
 파란 하늘 아래 모두 함께 떠나요 
 lets go summer 
 햇살이 부서지는 바닷가에 
 우리함께 달려가요 
 시원한 파도가 말해주는 시원한 여름 
 사랑하는 우리 영원히 사랑해 
 별이 쏟아지는 저 바다 위로 
 우리 함께 달려가요 
 저 푸른 바다로 가는 저 바다를 향해 
 손을 잡고 신나는 음악에 맞춰 
 행복을 느끼는 이 순간 
 저 바다가 보여 멋진 하늘이 보여 
 언제나 사랑을 비춰줘 빛나는 바다를 
 하얀 모래위에 예쁘게 적어 놓은 
 사랑해란 편지와 커다란 선물 
 푸른 바다로 신나는 여름 속으로 
 푸른 바다로 신나는 여름 속으로 
 언제나 함께 있어줘요 
 파란 하늘 아래 모두 함께 떠나요 
 lets go summer 
 햇살이 부서지는 바닷가에 
 우리함께 달려가요 
 시원한 파도가 말해주는 시원한 여름 
 사랑하는 우리 영원히 사랑해 
 별이 쏟아지는 저 바다 위에 
 우리함께 달려가요 
 저 푸른 바다로 가는 저 바다를 향해 
 손을 잡고 신나는 음악에 맞춰 
 행복을 느끼는 이 순간 
 저 바다가 보여 멋진 하늘이 보여 
 언제나 사랑을 비춰줘 빛나는 바다를 
 맑은바다를 바라보며 
 너와 난 바다를 가르고 싶었어 
 너의 마음을 내게줘 tell me baby 
 oh my lover 언제나 함께해 
 blue lover 언제나 함께있어 
 oh my love

In [ ]:
lyric_generator('댄스', '주말마다 강아지랑 산책해요<br> 강아지는 너무 귀엽고 <br> 폴짝 뛰어다녀요<br> 나는 행복해요')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 15263, 14304, 14971,  8202, 16780, 14088, 42005, 39847,
         14000, 21752, 42005,  7496,  8437, 16357, 22481,  8055, 42005, 14264,
         38380,  8055,     3]], device='cuda:0')
0: 주말마다 강아지랑 산책해요 
 강아지는 너무 귀엽고 
 폴짝 뛰어다녀요 
 나는 행복해요 
 내 손을 잡아 줄 사람 필요해 
 외롭게 하던 그 모든 것들을 
 나에 곁으로 와 바꿔준 이 사람 
 정말 고마워요 이 사랑을 믿어요 
 내 마음속 가득찬 그대를 향해 달려갈게요 
 모두 다 같이 떠나보자 하늘위로 
 우리 둘이 함께 하늘을 날아 가자 우리 둘이 함께 
 푸른 초원위에 그림 같은 집을 지어 살아봐요 
 가끔 눈이라도 오면 같이 걸어보고 
 멋진 밤에는 누구와 단 둘이 별을 보며 
 즐거운 상상을 해요 
 우리 모두 모여 하늘을 날아요 
 하늘 위로 
 파란하늘 아래서 자유롭게 살아봐요 
 항상 같은 하늘 아래서 함께해요 
 
 가끔 눈이라도 오면 같이 걸어보고 
 멋진 밤에는 누구와 단 둘이 별을 보며 
 즐거운 상상을 해요 
 우리 모두 모아 하늘을 날아요 
 하늘 위로 
 파란하늘 아래서 자유롭게 살아봐요 
 항상 같은 하늘 아래서 함께해요




In [ ]:
lyric_generator('발라드', '주말마다 강아지랑 산책해요<br> 강아지는 너무 귀엽고 <br> 폴짝 뛰어다녀요<br> 나는 행복해요')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42003, 15263, 14304, 14971,  8202, 16780, 14088, 42005, 39847,
         14000, 21752, 42005,  7496,  8437, 16357, 22481,  8055, 42005, 14264,
         38380,  8055,     3]], device='cuda:0')
0: 주말마다 강아지랑 산책해요 
 강아지는 너무 귀엽고 
 폴짝 뛰어다녀요 
 나는 행복해요 
 어떡하죠 
 눈물이나요 너무 미안해져요 
 내 맘이 그댈 향한 것만 같아 
 너무 고마워요 사랑하게 됐나봐 
 love you love you love you 
 안아 주고 싶어요 
 시간이 지나도 
 영원토록 헤어지지 말아요 
 love you love you love you 
 그대만을 원해요 그대의 사랑 
 언제까지나 영원히 
 love you love you love you 
 기다려요 우리 함께 할거죠 
 눈물나는 날에도 
 나 항상 그대 곁에 있고 싶어 
 사랑해 사랑해 
 어떡하죠 눈물이나요 
 너무 미안해져요 
 내 맘이 그댈 향한 것만 같아 
 너무 고마워요 사랑하게 됐나봐 
 love you love you love you 
 안아 주고 싶어요 
 시간이 지나도 
 영원토록 헤어지지 말아요 
 love you love you love you 
 그대만을 원해요 그대의 사랑 
 언제까지나 영원히 
 love you love you love you 
 기다려요 우리 함께 할거죠 
 눈물나는 날에도 
 나 항상 그대 곁에 있고 싶어 
 사랑해 사랑해 
 언제까지나 영원히 
 love you love you love you 
 기다려요 우리 함께 할거죠 
 눈물나는 날에도 
 나 항상 그대 곁에 있고 싶어




In [ ]:
# 생성된 가사가 거의 영어인 경우
lyric_generator('댄스', '토요일 멘토링은 즐거워')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 18751, 35709, 30048, 26576,     3]], device='cuda:0')
0: 토요일 멘토링은 즐거워 
 오 그대와 함께라면 날아갈 수 있어 펼쳐진 넓은 저 우주를 향해서 
 oh love me tell me why dont you know how come on baby 
 god is your favorite girl so 
 youre just mine baby right now 
 everybody come on come on lets dance 
 right now right now 
 its my shooting star 
 my shooting star now everybody head like headlike headlike now 
 yeah 
 you will never let me fall in 
 until the end of time we holdin together with no hands 
 were back again bring this back 
 you will never let me fall in 
 until the end of time we holdin together with no hands 
 were back again bring this back 
 yo here we go oh man im a party with you boogie to the floor 
 uknow i should feel it hot tarot bust your body rock 
 hit the mic pound it up pump it up 
 we gonna make it now lets dance 
 right now right now 
 its my shooting star 
 my shooting star now everybody head like he




In [ ]:
# 훅만 반복되는 경우
lyric_generator('발라드', 'I love you')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42003,  2049, 29808, 18641,     3]], device='cuda:0')
0: i love you 
 
 oh ah oh ah oh ah 
 
 wanna be yourself in the battle crazy like space around yeah yeah 
 
 파도 치는 바다를 헤엄쳐 
 그 속에 함께해 uh uh 
 파도치는 바다 위를 떠다닌 저 별들 중에 하나 
 
 i just hold my hand 
 
 i just dont know 
 
 i just see it 
 
 i just love me to think about you 
 
 we are all time getting party time 
 
 우리의 기쁨을 찾아주는 이 밤하늘에 달빛 
 
 i love you baby 
 
 oh ah oh ah oh ah ah 
 
 yeah yeah yeah yeah




In [ ]:
# 입력한 글은 안나오게 하기
lyric_generator('댄스', '주말마다 강아지랑 산책해요<br> 강아지는 너무 귀엽고 뛰어다녀요')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 15263, 14304, 14971,  8202, 16780, 14088, 42005, 39847,
         14000, 21752, 16357, 22481,  8055,     3]], device='cuda:0')
 
 이제는 때가 됐죠 
 i love you love you love you baby 
 love you love you love you baby 
 love you love you love you baby 
 내 곁에서 더 높이 날아갈래요 
 그대 이제 그만 좀 뛰놀까요 
 baby baby baby 내 맘에 들어와요 
 baby baby baby 날 안아줘요 
 내 눈을 바라봐요 
 내 손을 잡아요 내게 맡겨봐요 
 baby baby baby 내게 와요 
 baby baby baby 날 사랑해줘요 
 언제나 함께 해요 
 i want your love 
 i need your love 
 i need your love 
 i need your love 
 내 곁에선 더 높이 날아갈래요 
 그대 이제 그만 좀 뛰놀까요 
 baby baby baby 내 맘에 들어와요 
 baby baby baby 날 안아줘요 
 내 눈을 바라봐요 
 내 손을 잡아요 내게 맡겨봐요 
 baby baby baby 내게 와요 
 baby baby baby 날 사랑해줘요 
 언제나 함께 해요 
 가끔은 혼자도 괜찮아 둘이어도 괜찮아 
 난 괜찮으니까 날 안아줘요 
 내 눈을 바라봐요 
 내 손을 잡아요 내게 맡겨봐요 
 baby baby baby 내게 와요 
 baby baby baby 날 사랑해줘요 
 언제나 함께 해요 
 baby baby baby 내게 와요 
 baby baby baby 날 안아줘요 
 내 눈을 바라봐요 
 내 손을 잡아요 내게 맡겨봐요 
 baby baby baby 내게 와요 
 baby baby baby 내 사랑 받아줄래요 
 내 눈 속에 너를 담아 보낼게 
 love forev




In [ ]:
lyric_generator('댄스', '토요일 멘토링은 즐거워')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 18751, 35709, 30048, 26576,     3]], device='cuda:0')
0: 토요일 멘토링은 즐거워 
 조금씩 너를 향해 가고 있어 이젠 모든 것을 뒤로한 채 널 향해 달려가고 있어 
 어떤 어려운 것도 다 상관없어 그저 사랑하는 너의 손길만 
 나를 위해 준비된 오직 단 한사람만을 위한 시간이란걸 느끼며 
 자 시작해 볼까 내 자신을 돌아봐봐 너의 모든 사랑이 기다리는 곳에 다가서 봐 오 오 
 
 우리 서로 멀리 있어도 좋아 떠나자 지금 자유로워 이 기분을 느껴봐 
 
 이끌리는 대로 날개를 펴 세상을 날아가도 괜찮아 원하는 곳으로 
 꿈을 찾아 떠나보는 거야 baby one two 
 
 시원한 바람결에 두 팔을 벌리고




In [ ]:
lyric_generator('댄스', '토요일 멘토링은 신나')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 18751, 35709, 30048, 29863,     3]], device='cuda:0')
0: 토요일 멘토링은 신나 
 
 
 
 hey hey hey lets go 
 
 파란 하늘 위엔 멋진 별이 떠 있어요 
 
 
 
 우린 높이 날아갈 수 있어 우릴 따라와 줘 
 
 
 lets start this party 
 
 우리 모두 신나게 yeah we come on 
 
 이제 시작인데 lets bring it right now 
 
 소리쳐줘 크게 외쳐봐 
 
 
 
 푸른하늘 위를 달려보자 시원한 바람 불어와 
 
 
 hey hey hey lets go 
 
 하얀 파도위에 떠오른 태양이 눈부셔 
 
 
 
 저기 멀리 빛나는 저 별을 바라봐 
 
 
 
 푸른 바다속에 뛰어들어 저 깊은 밤을 날아봐 
 
 
 
 
 lets start this party 
 
 우리 모두 신나게 yeah we come on 
 
 이제 시작인데 lets bring it right now 
 
 소리쳐줘 크게 외쳐봐 
 
 
 
 푸른하늘 위를 달려보자 시원한 바람 불어와 
 
 
 
 hey hey hey lets go 
 
 하얀 파도위에 떠오른 태양이 눈부셔 
 
 
 
 저기 멀리 빛나는 저 별을 바라봐 
 
 
 
 푸른 바다속에 뛰어들어 저 깊은 밤을 날아봐 
 
 
 
 소리쳐줘 크게 외쳐봐 
 
 
 푸른 하늘 위를 달려보자 시원한 바람 불어와 
 
 
 
 하나 둘 셋 lets go 
 
 lets start this party 
 
 우리 모두 신나게 떠나볼래 
 
 
 
 푸른 바다 위로 띄워줄래 
 
 
 
 our summer fever 
 
 summer love together




In [ ]:
lyric_generator('댄스', '토요일 멘토링은 신나')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 18751, 35709, 30048, 29863,     3]], device='cuda:0')
0: 토요일 멘토링은 신나 
 오늘 밤엔 그대와 나만의 battle group star 
 그대와 단둘이 떠나요 둘이서 우리 단둘이 
 푸른 바다 위에 fishbacka bikini 
 이 밤이 새도록 너와 함께 
 뜨거운 태양아래 fishbacka bikini 
 내 맘을 모두 줄게 yeah 
 i wanna love you boy ooh ooh ooh 
 i wanna love you boy ooh ooh ooh 
 you love me love me boy 
 i love you boy love me boy 
 heartbeat heartbeat boy 
 heartbeat heartbeat boy 
 나를 한번 봐줘요 heartbeat heartbeat 
 heartbeat heartbeat boy 
 내 심장은 두근두근 뛰죠 난 매일이 생일인데 
 오늘만은 그대와 나뿐인 걸요 oh my lover 
 그대는 나의 shinee world 
 파란하늘 아래 sunshine 
 태양은 우릴 향해 비추고 
 달빛은 우릴 향해 비춰 
 우리 둘만 있는 그곳으로 oh my lover 
 우리의 별이 빛나는 이유는 
 우리만의 love story 
 이 밤이 새도록 너와 함께 
 뜨거운 태양아래 fishbacka bikini 
 이 밤이 새도록 너와 함께 
 뜨거운 태양아래 fishbacka bikini 
 내 맘을 모두 줄게 yeah 
 i wanna love you boy ooh ooh ooh 
 i




In [ ]:
# gpt_2
lyric_generator('댄스', '매일 강아지랑 산책해')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 14759, 14971,  8202, 16780,  8010,     3]],
       device='cuda:0')
0: 매일 강아지랑 산책해 
 ayo 
 youre goin on and on and on and on 
 we going going on and on and on 
 hey baby 
 너 따위 버릴래 완전히 짜증나 미칠 것 같애 
 woo 난 너를 못 잊어 정말로 너무 미워 죽겠어 
 내 맘은 왜 말을 안 듣는거니 왜 자꾸 넌 왜 계속 왜 
 내 맘대로 할 수 없니 now let it go let it go let it go 
 hey now baby now 가지마 ride ride ride yeah 
 내 앞에선 꼼짝 하지마 woo waha oh baby 
 너 따윈 버렸어 like a super star star star 
 i got you boy woo waha i got you 
 boy 넌 너 따윈 버렸어 like a super star star star 
 내 맘대로 할 수 없어 bye bye bye bye 
 youre goin on and on and on and on 
 gonna get a danger danger 
 네 가슴속에 깊은 상처는 더 깊어 가 
 woo 난 이제 너 따윈 지겨워졌어 정말 어떡하라고 
 널 떠나버려 나 같은 건 필요 없어 woo waha oh baby 
 너 따윈 버렸어 like a super star star 
 i got you boy waha i got you 
 boy 넌 너 따윈 버렸어 like a super star star star 
 내 맘대로 할 수 없어 bye bye bye bye 
 youre goin on and on and on and on 
 gonna get a d




In [ ]:
lyric_generator('댄스', '너무 피곤해')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 14000, 18262,  8010,     3]], device='cuda:0')
내가 포기


'내가 포기'

In [ ]:
lyric_generator('발라드', '너무 피곤해')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42003, 14000, 18262,  8010,     3]], device='cuda:0')



''

In [ ]:
lyric_generator('발라드', '너무 피곤해')

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42003, 14000, 18262,  8010,     3]], device='cuda:0')
부디 행복해


'부디 행복해'

In [ ]:
if re.sub(r'[a-z\n]','',a.split('<br>')[-1]).strip() == "" :
  b = a.split('<br>')[:-1]
  c = ""
  for i in b:
    if i == ' ':
      i = re.sub(r" ", "\n \n", i)
    c += i
    c = re.sub(r"  ", "\n", c)
print(c)

0: 매일 강아지랑 산책해 
 
 lets go start lets go start lets go start start
lets go start lets go start start start
hey we are the maestro
yeah i like it like that 
 
 we are the maestro
yeah i like it like that ah
ohohohoh uh uh uh uh huh ahh uh huh uh uh huh ah huh 
 
 we are the maestro
yeah i like it like that ah
ohohohoh uh uh uh huh ah huh ah
uh huh huh uh huh ah ah huh ah huh 
 
 항상 함께 했었잖아 정말 사랑했었어
우리 둘만의 세상에서 행복하게만 지내자고 말했었잖아
그렇게 쉽게 떠나갈 거면서
그 많은 시간 동안 나는 많이도 다투고 들게 상처 받아 받았어
이제는 그만해 다신 울지 않을래 너와 나 함께라면 
 
 we are the maestro
yeah i like it like that
ohohohoh uh uh uh huh ahh uh huh ah huh 
 
 we are the maestro
yeah i like it like that ah 


In [ ]:
re.sub(r'[a-z\n]','',a.split('<br>')[-1]).strip()

''

In [ ]:
a.find('<br>')

15

In [ ]:
b = a.split('<br>')[:-1]
b

['0: 매일 강아지랑 산책해 ',
 ' ',
 ' lets go start lets go start lets go start start ',
 ' lets go start lets go start start start ',
 ' hey we are the maestro ',
 ' yeah i like it like that ',
 ' ',
 ' we are the maestro ',
 ' yeah i like it like that ah ',
 ' ohohohoh uh uh uh uh huh ahh uh huh uh uh huh ah huh ',
 ' ',
 ' we are the maestro ',
 ' yeah i like it like that ah ',
 ' ohohohoh uh uh uh huh ah huh ah ',
 ' uh huh huh uh huh ah ah huh ah huh ',
 ' ',
 ' 항상 함께 했었잖아 정말 사랑했었어 ',
 ' 우리 둘만의 세상에서 행복하게만 지내자고 말했었잖아 ',
 ' 그렇게 쉽게 떠나갈 거면서 ',
 ' 그 많은 시간 동안 나는 많이도 다투고 들게 상처 받아 받았어 ',
 ' 이제는 그만해 다신 울지 않을래 너와 나 함께라면 ',
 ' ',
 ' we are the maestro ',
 ' yeah i like it like that ',
 ' ohohohoh uh uh uh huh ahh uh huh ah huh ',
 ' ',
 ' we are the maestro ',
 ' yeah i like it like that ah ']

In [ ]:
c = ""
for i in b:
  if i == " ":
    i = re.sub(r" c", "\n", i)
  c += i
  c = re.sub(r"  ", "\n", c)
print(c)

0: 매일 강아지랑 산책해 
 lets go start lets go start lets go start start
lets go start lets go start start start
hey we are the maestro
yeah i like it like that 
 we are the maestro
yeah i like it like that ah
ohohohoh uh uh uh uh huh ahh uh huh uh uh huh ah huh 
 we are the maestro
yeah i like it like that ah
ohohohoh uh uh uh huh ah huh ah
uh huh huh uh huh ah ah huh ah huh 
 항상 함께 했었잖아 정말 사랑했었어
우리 둘만의 세상에서 행복하게만 지내자고 말했었잖아
그렇게 쉽게 떠나갈 거면서
그 많은 시간 동안 나는 많이도 다투고 들게 상처 받아 받았어
이제는 그만해 다신 울지 않을래 너와 나 함께라면 
 we are the maestro
yeah i like it like that
ohohohoh uh uh uh huh ahh uh huh ah huh 
 we are the maestro
yeah i like it like that ah 


In [ ]:
remove_lastline(lyric_generator('댄스', '매일 강아지랑 산책해'))

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 14759, 14971,  8202, 16780,  8010,     3]],
       device='cuda:0')
True
False
0: 매일 강아지랑 산책해 
 게임 초대는 안 해줘도 좋아 
 하루 종일 니 생각을 할 텐데 
 하루 종일 니 생각만 할 텐데 
 이런 게 바로 보나 마나 보나마나 보나마나 born to my world 
 when i fall in love with you 
 내가 너에게 다가가도 
 아무 말하지 않을게 
 그저 바라보는 것도 바라지도 않을게 
 its just for you 
 내가 너를 처음 봤을 때 
 네 생각 그대로 다 똑같았었는데 
 when i fall in love with you 
 조금 더 가까워지면 괜찮을 것 같아 
 매일매일 밤 널 보고 싶어 
 매일 너를 생각해 언제나 너의 곁에서 
 널 따라다닌 시간들을 생각하면 눈물이 나 
 내가 만약 이렇게 
 라면 정말 사랑할 수 있을까 
 love love love oh babe 
 love love love oh babe 
 매일 사랑해 이 노랠 들으며 함께해 
 love love love love oh babe 
 love love love love oh babe 
 하루 종일 니 생각을 할 텐데 
 하루 종일 니 생각만 할 텐데 
 이런 게 바로 보나 마나 보나마나 보나마나 born to my world 
 when i fall in love with you 
 내가 너에게 다가가도 아무 말하지 않을게 
 그저 바라보는 것도 바라지도 않을게 
 its just for you 
 내가 너를 처음 봤을 때 
 네 생각 그대로 다 똑같았었는데 
 when i fall in love with you 
 조금 더 가까워지면 괜찮을 것 같아 
 매일매일 밤 널 보고 싶어 
 매일 너를 생각해 언제나 너의 곁에서 
 널 따라다닌 시간들을 생각하면 눈물이 나 
 내가 만약 이렇게 
 라면 정말 사랑할 수 있을까 
 love 

In [ ]:
a = lyric_generator('댄스', '매일 강아지랑 산책해')
a

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 14759, 14971,  8202, 16780,  8010,     3]],
       device='cuda:0')


'0: 매일 강아지랑 산책해 <br> 어려운 얘기 일단 좀 해 둬 볼까 <br> 너한테만 물어봐 <br> 나 지금 몇 달째인지 <br> 우리 엄마 말고 다른 아빠한테 물었어 <br> 아빠도 몰라 그냥 궁금해 <br> 날 좋아하긴 하는 건지 뭔지 <br> 나도 알 수 없지만 난 좋아 <br> 자꾸 나를 귀찮게 만들어 자꾸 너를 <br> 내 곁에만 있게 해 줘 love you girl love you love you <br> love you girl 날 지켜줘 <br> i want you only you girl <br> love you girl love you love you <br> love you girl 날 지켜줘 <br> i want you only you girl <br> love you girl love you love you <br> love you girl <br> 내가 너를 지켜 줄 테니 <br> i gotta have a daddy heart <br> 이젠 나도 너를 지켜 <br> 널 안아줄 거야 oh yeah <br> 내가 좀 더 다가갈게 <br> love you girl love you love you <br> love you girl 날 지켜줘 <br> i want you only you girl <br> love you girl love you love you <br> love you girl 날 지켜줘 <br> i want you only you girl <br> love you girl love you love you <br> love you girl <br> 지금 나는 어디쯤 있는 걸까 <br> 점점 너에게로 가고 있어 <br> 사랑한다고 말해주지만 <br> 내게만 집중해 what you know <br> 왜 넌 나를 헷갈리게 해 계속 나를 <br> 귀찮게 만들어 자꾸 너를 <br> 내게만 있게 해 줘 love you girl <br> love you girl 날 지켜줘 <br> i want you only you gir\n\n'

In [ ]:
print(a.replace('<br>','\n'))

0: 매일 강아지랑 산책해 
 항상 지켜왔던 우리들의 소중한 만남이 
 이젠 조금씩 하나 둘 자리 잡았어 
 항상 외롭기만 했던 나를 위해서 
 넌 항상 밝은 모습만을 보였지 
 나의 곁에 있으면 함께 있는 것처럼 
 그 눈빛을 보면 나는 너를 알아봤어 
 언제나 너와 함께 있을거라던 나의 믿음은 
 나의 마음 속에 영원히 남았는걸 
 oh baby youre my love 
 이제 난 더 이상 혼자가 아니야 
 oh baby youre my love 
 너의 모든 걸 사랑꺼야 언제나 이대로 내 안에 
 너를 영원히 간직해 
 my love my life 
 서로 사랑하는걸 잊진 말아줘 
 love my life 
 너 하나만 바라보고 널 사랑해 
 my love my life 
 영원히 너를 위해 아껴줄께 
 love my life 
 love my life 
 언제나 내가 먼저 다가가 지켜 줄 꺼야 
 오직 내 사랑은 언제까지나 
 나만의 너를 위해 언제까지 아껴줄께 
 oh baby youre my love 
 이제 난 더 이상 혼자가 아니야 
 oh baby youre my love 
 너의 모든 걸 사랑꺼야 언제나 이대로 내 안에 
 너를 영원히 간직해 
 my love my life 
 서로 사랑하는걸 잊진 말아줘 
 love my life 
 너 하나만 바라보고 널 사랑해 
 my love my life 
 영원히 너를 위해 아껴줄게 
 my love my life 
 이제는 너를 향한 내 마음 모두다 
 사랑하기 때문에 영원히 사랑해 
 너의 모든 걸 사랑꺼야 언제나 이대로 내 안에 
 너를 영원히 간직해 
 my love my life 
 서로 사랑하는걸 잊진 말아줘 
 love my life 
 너 하나만 바라보고 널 사랑해 
 my love my life 
 영원히 너를 위해 아껴줄께 
 love my life 
 love my life 
 너 하나만 바라보고 널




In [ ]:
def precessing(a):
  # 마지막 줄에 영어만 있다면,
  if re.sub(r'[a-z\n]','',a.split('<br>')[-1]).strip() == "" :
    b = a.split('<br>')[:-1]
    c = ""
    for i in b:
      if i == ' ':
        i = re.sub(r" ", "\n \n", i)
      c += i
      c = re.sub(r"  ", "\n", c)
      output = c
  else:
    output = a.replace('<br>','\n')
  
  count = []
  count2 = []
  for word in output:
    if re.sub(r'[a-z]','',word) == "":
      count += word
    elif word == " ":
      count2 += word
  if len(count)/(len(output)-len(count2)) > 0.6:
    output = "버튼을 다시 눌러주세요."
  print(output)
  return 
precessing(a)

0: 매일 강아지랑 산책해
어려운 얘기 일단 좀 해 둬 볼까
너한테만 물어봐
나 지금 몇 달째인지
우리 엄마 말고 다른 아빠한테 물었어
아빠도 몰라 그냥 궁금해
날 좋아하긴 하는 건지 뭔지
나도 알 수 없지만 난 좋아
자꾸 나를 귀찮게 만들어 자꾸 너를
내 곁에만 있게 해 줘 love you girl love you love you
love you girl 날 지켜줘
i want you only you girl
love you girl love you love you
love you girl 날 지켜줘
i want you only you girl
love you girl love you love you
love you girl
내가 너를 지켜 줄 테니
i gotta have a daddy heart
이젠 나도 너를 지켜
널 안아줄 거야 oh yeah
내가 좀 더 다가갈게
love you girl love you love you
love you girl 날 지켜줘
i want you only you girl
love you girl love you love you
love you girl 날 지켜줘
i want you only you girl
love you girl love you love you
love you girl
지금 나는 어디쯤 있는 걸까
점점 너에게로 가고 있어
사랑한다고 말해주지만
내게만 집중해 what you know
왜 넌 나를 헷갈리게 해 계속 나를
귀찮게 만들어 자꾸 너를
내게만 있게 해 줘 love you girl
love you girl 날 지켜줘 


In [ ]:
precessing(lyric_generator('댄스', '매일 강아지랑 산책해'))

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


tensor([[42000, 42004, 14759, 14971,  8202, 16780,  8010,     3]],
       device='cuda:0')
0: 매일 강아지랑 산책해
hey hey hey hey hey hey
우리 모두 응원해요
우리 모두 응원해요
함께해요 lets go now
자 우리 모두 모여봐요
내 손을 잡고 한번 뛰어봐
우리 앞에 있는 모든 것들을 이뤄줄게
하나 되는 그 날까지 oh oh oh oh
푸른 하늘처럼 맑은 눈빛으로
푸른 파도처럼 밀려와
우리 앞에 있는 모든 것을 이뤄줄게
푸른 바다처럼 맑은 눈빛으로
푸른 물결처럼 밀려와
우리 앞에 있는 모든 것을 이뤄줄게
hey hey hey hey hey
우리 모두 응원해요
우리 모두 응원해요
함께해요 lets go now
자 우리 모두 모여봐요
내 손을 잡고 한번 뛰어봐
우리 앞에 있는 모든 것들을 이뤄줄게
하나 되는 그 날까지 oh oh oh oh
푸른 하늘처럼 맑은 눈빛으로
푸른 파도처럼 밀려와
우리 앞에 있는 모든 것을 이뤄줄게
푸른 바다처럼 맑은 눈빛으로
푸른 물결처럼 밀려와
우리 앞에 있는 모든 것을 이뤄줄게
hey hey hey hey
lets go now here we go yeah 


In [ ]:
count = []
count2 = []
for word in output:
  if re.sub(r'[a-z]','',word) == "":
    count += word
  elif word == " ":
    count2 += word
if len(count)/(len(output)-len(count2)) > 0.5:
  print("버튼을 다시 눌러주세요.")

버튼을 다시 눌러주세요.


In [ ]:
output[17]

'o'

In [ ]:
len(count), len(output)-len(count2)

(245, 525)